# Внутренние устройства типов данных для математических вычислений Python, Numpy, Pytorch.

**Статья основана на второй главе этой <a href="https://github.com/jakevdp/PythonDataScienceHandbook">книги</a> (Introduction to NumPy), но дополнена уточнениями и расширенна главой про broadcasting. Книжка очень хорошая. :-)**

## Подключение библиотек

In [ ]:
non_removable_variables = []
non_removable_variables.append('non_removable_variables')

In [ ]:
from simple_benchmark import benchmark
non_removable_variables.append('benchmark')

import torch
non_removable_variables.append('torch')

import time
non_removable_variables.append('time')

import numpy as np
non_removable_variables.append('np')

import sys
non_removable_variables.append('sys')

import gc
non_removable_variables.append('gc')

In [ ]:
def clear_memory():
    variables = %who_ls
    for var_name in variables:
        if var_name not in non_removable_variables:
            if callable(globals()[var_name]) == True:
                continue
            else:
                print(var_name)
                del globals()[var_name]
    
    print('collected count ' + str(gc.collect()))

## Введение 

При анализе данных приходится иметь дело с числовыми массивами огромных размеров.  Для того, чтобы математические операции занимали минимальное количества времени и памяти необходимо понимание внутреннего устройства структур, использующихся для хранения и манипуляций с ними.  

## Типы данных в Python

Python в отличии от С++ и Java является не строго типизированном языком программирования, что дает ему огромную гибкость. 

**Пример**

```c++
/* C code */
int result = 0;
for(int i=0; i<100; i++){
    result += i;
}
```

```python
# Python code
result = 0
for i in range(100):
    result += i
```

Ключевое отличие в том, что в С++ тип каждой переменной определяется при ее создании и не может быть изменен в дальнейшим. В Python же тип переменной определяется динамически, во время выполнения. Это означает, что мы можем присвоить переменной данные любого типа, меняя при этом тип самой переменной.  

**Пример**

In [ ]:
var = 3; print(type(var)) #Создали переменную типа int 

In [ ]:
var = '3'; print(type(var)) #Меняем значения и тип переменной 

На С++ такие манипуляции просто невозможны.

```c++
/* C code */
int result = 0;
result = "Ноль"; # Вызовет ошибку так, как result не строка, а число  
```

Понимание этого очень важно для эффективной манипуляции данными в data science.

## Внутренние устройство типа Integer в Python

Python написан на языке С и каждый тип данных в нем является классом в Python.

In [ ]:
print(type(0)); print(type(0.8)); print(type("ooo"));

В то время как на стороне С каждый тип приставляет из себя структуру, содержащую набор полей.

Итак, **var = 10** не просто переменная типа integer, это указатель на структуру языка С. Рассмотрим подробно эту структуру. 

```c++
struct _longobject {
    long ob_refcnt;
    PyTypeObject *ob_type;
    size_t ob_size;
    long ob_digit[1];
};
```

- Поле ob_refcnt - счетчик ссылок, необходим Python для управления памятью 
- Поле ob_type - кодирует тип (integer в нашем случаи)
- Поле ob_type - количество элементов в массиве ob_digit (необходим для длинной арифметики. Подробно описан <a href="https://www.codementor.io/@arpitbhayani/how-python-implements-super-long-integers-12icwon5vk">здесь</a>)
- Поле ob_type - непосредственно наши данные 

Именно благодаря этим дополнительным полям  Python является не строго типизированным языком.

Если сравнивать integer в С++ с integer в python, вырисовывается следующая картина. 

![cint_vs_pyint](images/type_of_datas_and_broadcasting/cint_vs_pyint.png)

Здесь PyObject_HEAD — структура, содержащая все ранее упомянутые поля: ob_refcnt, ob_type и т. д. 

### Вывод

В С++ integer — это просто указатель на область памяти, содержащий значении переменной, тогда как в Python  integer — указатель на область памяти, содержащую Python объект, который помимо значения переменной содержит множество дополнительных полей. Такое представление переменных избыточно и особенно это будет заметно когда таких переменных будет тысячи, а то и десятки тысячи.

## Внутреннее устройства типа list в Python 

Допустим у нас есть dataset с числами и мы решили заняться его анализом. Первое, что нужно сделать — это выбрать структуру данных для их хранения. Давайте разберемся почему список не лучшее решения. 

Заполнить список числами можно так:

In [ ]:
L = [1,2,3,4]

In [ ]:
type(L[0])

или так

In [ ]:
L = ['1','2','3','4']

In [ ]:
type(L[0])

Либо даже так

In [ ]:
L = [1,'2', 0.6, True]

In [ ]:
[print(type(l)) for l in L]

Видно, что список поддерживает динамическую типизацию. Помимо структуры описывающей тип списка: счетчика ссылок, длины и т. д. он также хранит указатели на каждый элемент, а элемент в свою очередь, является Python объектом со своим счетчиком ссылок, длиной и т. д. 

![list](images/type_of_datas_and_broadcasting/List.png)

### Вывод

Если мы хотим анализировать данные определенного типа, то структура где каждый элемент хранит свой тип избыточна. Гораздо выгодней хранить общий тип всех элементов отдельно. Помимо этого, элементы списка разбросаны по памяти и каждый раз, обращаясь к элементу, мы берем его указатель, переходим по адресу хранящимся в нем, извлекаем объект и лишь в конце получаем значение. Такой способ излечения значения будет серьезно тормозить программу, анализирующую большие данные.

Давайте попробуем устранить указанный недостатки.

## Python array

Начиная с версии 3.3 в Python появился встроенный тип данных array, который посути является обверткой над массивами языка С. Рассмотрим его структуру. 

![python_array](images/type_of_datas_and_broadcasting/python_array.png)

Все элементы в нем имеют одинаковый тип, который задается при создании объекта и не может быть изменен в дальнейшим

In [ ]:
import array

In [ ]:
L = [1,2,3,4]

In [ ]:
arr = array.array('i',L) 

In [ ]:
arr

Здесь i — это код типа данных языка С (integer). Также доступны другие типы (см. <a href="https://docs.python.org/3/library/array.html">здесь</a>)

```python
arr[0] = 'l' # Вызовет ошибку неверный тип 
````

Благодаря тому, что общий тип храниться в одном месте экономиться значительное количество памяти. 

In [ ]:
L = list(range(1000))
A = array.array('i', L)

In [ ]:
print(type(L[0])); print(type(A[0])); # Типы идентичны 

In [ ]:
sys.getsizeof(L)

In [ ]:
sys.getsizeof(A)

In [ ]:
sys.getsizeof(L) / sys.getsizeof(A) # Меньше почти в 2 раза 

Может показаться, за счет того, что данные хранятся в упакованном виде, значительно быстрее происходит извлечения значения из памяти, но это не так.

In [ ]:
%%timeit 
result = 0
for i in A:
    result += i 

In [ ]:
%%timeit 
result = 0
for i in L:
    result += i

Чтобы найти причину задержки, обратимся к исходному коду функций извлекающих значения. 

Функция списка 

```C++
PyObject *
PyList_GetItem(PyObject *op, Py_ssize_t i)
{
    /* error checking omitted */
    return ((PyListObject *)op) -> ob_item[i];
}
```

Функция массива

```C++
static PyObject *
l_getitem(arrayobject *ap, Py_ssize_t i)
{
    return PyLong_FromLong(((long *)ap->ob_item)[i]);
}
```

В отличии от списка, при извлечении значения из массива на стороне С создается Python объект. Это абсолютно необходимо, так как Python может работать только с объектами. Именно эта операция так сильно тормозит массивы. (Подробнее <a href=»https://stackoverflow.com/questions/36778568/why-are-pythons-arrays-slow»>здесь</a>)

Еще один недостаток связан с переполнением. В типах с фиксированным размерам нет длинной арифметики.  

In [ ]:
max_value = 2_147_483_647

In [ ]:
A[0] = max_value; L[0] = max_value 

A[0] += 1 # Вызовет ошибку переполнения 

In [ ]:
L[0] += 1 # Длинная арифметика, поле ob_size увеличилось на единицу. 

In [ ]:
L[0]

### Вывод

Для эффективной манипуляции числовыми данными в Python, мало иметь структуру данных с фиксированном типом и непрерывным размещением значений, еще нужно все математические операции выполнять на стороне С, не создавая  Python объекты.

## Numpy ndarray

Специально для манипуляции с массивными данных был разработан ndarray, который входит в состав модуля numpy. Его внутренняя структура очень похожа на структуру типа array. 

![ndarray](images/type_of_datas_and_broadcasting/ndarray.jpeg)

Следовательно он занимает примерно столько же памяти, как и array.

In [ ]:
L = list(range(10_000))
A = array.array('i', L)
NA = np.array(L, np.int32)

In [ ]:
print(sys.getsizeof(A)); print(sys.getsizeof(NA))

Нет длинной арифметики. 

In [ ]:
type(NA[0])

In [ ]:
np.iinfo(np.int32)

In [ ]:
NA[0] = 2147483647 + 1 # Переполнение 

In [ ]:
NA[0]

Но все математические операции реализованы на стороне С, что ведет к многократному ускорению вычислений. Проедем benchmark тест 

In [ ]:
list_of_lists = []

In [ ]:
for i in range(6):
    list_of_lists.append(list(range(10**(i + 1))))

In [ ]:
list_of_arrays = []

In [ ]:
for i in range(len(list_of_lists)):
    list_of_arrays.append(array.array('i', list_of_lists[i]))

In [ ]:
list_of_np_arrays = []

In [ ]:
for i in range(len(list_of_lists)):
    list_of_np_arrays.append(np.array(list_of_lists[i], np.int32))

In [ ]:
def list_sum(i):
    return sum(list_of_lists[i])

In [ ]:
def array_sum(i):
    return sum(list_of_arrays[i])

In [ ]:
def np_array_python_sum(i):
    return sum(list_of_np_arrays[i])

In [ ]:
def np_array_sum(i):
    return list_of_np_arrays[i].sum()

In [ ]:
funcs = [list_sum, array_sum, np_array_python_sum, np_array_sum]
arguments = {10**(i + 1): i for i in range(len(list_of_lists))}
argument_name = 'elements count'
aliases = {list_sum: 'list sum', array_sum: 'array sum', np_array_python_sum: 'numpy sum in loop', np_array_sum: 'function in C++'}
b = benchmark(funcs, arguments, argument_name, function_aliases=aliases)

In [ ]:
b.plot()

C Python функциями numpy работает медленней чем все остальные типы. Причина какая же как в array - создание python объекта при извлечении значения. 

Примерно до 10000 элементов функция, выполняющаяся на С++, имеет константное время выполнение и работает медленней всех, а потом уверенно лидирует. Это объясняется скоростью создания ndarray-я. 

In [ ]:
L = list(range(10_000))
A = array.array('i', L)
NA = np.array(L)

In [ ]:
len(L)

In [ ]:
%%timeit 
sum(L)

In [ ]:
%%timeit 
sum(A)

In [ ]:
%%timeit 
sum(NA)

Финальный тест

In [ ]:
%%timeit 
NA.sum() # Выполняется в С++

Просто за счет того, что функция написана на С++, мы получили примерно четырех кратный прирост производительности. 

### Вывод

Numpy был разработан специально для манипуляции большими массивами данных. Все математические операции для ndarray реализованы на С. Из — за создания Python объектов циклы с эти типами данных работают очень медленно и следует их избегать. 

Может показаться что существенный прирост производительности можно добиться распараллелив ndarray и применив векторизацию (SIMD инструкции). В следующей главе мы подробно изучим этот вопрос.

In [ ]:
clear_memory()

## Распараллеливаем Numpy ndarray

<a href="https://scipy-cookbook.readthedocs.io/items/ParallelProgramming.html">Ссылка на источник</a>

В Numpy реализована поддержка библиотек линейной алгебры, которые в свою очередь реализуют SIMD инструкций и много поточность (см <a href="https://habr.com/ru/post/274331/">здесь</a>). Можно посмотреть какую версию библиотеки поддерживает локальна версия numpy. 

In [ ]:
print(np.__config__.show())

В моем случаи это openblas

Следовательно реализованные функции для линейной алгебры будут работать параллельно из коробки (см <a href="https://numpy.org/doc/stable/reference/routines.linalg.html">список функций</a>)

In [ ]:
size = 5000

In [ ]:
a = np.random.rand(size,size)
b = np.random.rand(size,size)

In [ ]:
%%timeit 
a @ b

![blas_multiprosses_op](images/type_of_datas_and_broadcasting/blas_multiprosses_op.jpg)

А обычные операции нет 

In [ ]:
%%timeit
a * b

![without_paralells](images/type_of_datas_and_broadcasting/without_paralells.jpg)

### Вывод

В Numpy используется масса приемов ускоряющих математические операции. В частности реализована поддержка библиотек линейной алгебры. Дальнейшего прироста производительности на CPU добиться очень сложно, но можно использовать GPU. В следующий главе мы ускорим вычисления с помощью Pytorch.

In [ ]:
clear_memory()

## Pytorch для ускорения математических операций 

Pytorch не использует numpy array, вместо этого в библиотеке реализован свой тип torch.tensor. Структура которого очень похожа на структуру  numpy array. Также как и в Numpy, в Pytorch реализована поддержка библиотек линейной алгебры, но сами поддерживаемые библиотеки могут отличаться.

In [ ]:
print(torch.__config__.show())

Так, как эти библиотеки можно настраивать, в частности numpy можно пересобрать с поддержкой других библиотек, нет никакого смысла сравнивать скорость работы torch.tensor с numpy array на CPU. Ключевое различие этих тензоров состоит в том, что torch.tensor может работать на GPU, эта возможность многократно увеличивает его производительность.

In [ ]:
A = np.random.rand(3000,3000)

In [ ]:
B = np.random.rand(3000,3000)

In [ ]:
start = time.time()
C = A @ B
end = time.time()

In [ ]:
cpu_time = end - start; print(cpu_time)

In [ ]:
device = torch.device('cuda')

In [ ]:
device

In [ ]:
A_gpu = torch.from_numpy(A).to(device)

In [ ]:
B_gpu = torch.from_numpy(B).to(device)

In [ ]:
c = A_gpu @ B_gpu # cuda warmup
torch.cuda.synchronize()

In [ ]:
start = time.time()
c = A_gpu @ B_gpu
torch.cuda.synchronize()
end = time.time()

In [ ]:
gpu_time = end - start; print(gpu_time)

In [ ]:
cpu_time / gpu_time

### Вывод

За счет того что вычисления производились на gpu, мы получили выигрыш во времени почти в три раза. Однако в использовании gpu для вычислений есть свои тонкости, в частности тратится большое количество времени на копирование памяти с хоста на устройства и обратно. Позднее мы  проведем benchmark тест и сравним работу на gpu с работой на cpu.

In [ ]:
%reset -f

## Broadcasting (Правила broadcasting-а для Pytorch идентичны)

<a href="https://numpy.org/doc/stable/user/basics.broadcasting.html">Ссылка на источник</a>

In [ ]:
import numpy as np

In [ ]:
A = np.array([[1,2,3],[4,5,6],[7,8,9]])

In [ ]:
B = np.array([[1,2,3],[4,5,6],[7,8,9]])

Ясно что вся мощь параллелизма будет работать когда массивы имеют одинаковый размер 

In [ ]:
A + B

In [ ]:
A - B

In [ ]:
A * B # Поэлементное умножение 

In [ ]:
A @ B # Матричное умножение 

Но что делать если нам нужно к каждому элементу матрицы добавить (отнять, поделить, сравнить) число ?

Но что делать если нам нужно к каждому элементу матрицы добавить (отнять, поделить, сравнить) число ? Неужели придется создавать массив нужной размерности и использовать его как операнд ? На помощь приходит broodcasting. При A + 1 — такой записи операция будет выполняться сама без расхода дополнительной памяти  

In [ ]:
A + 1

In [ ]:
A - 1

In [ ]:
A > 1

In [ ]:
A < 5

Также работает с векторами 

In [ ]:
v = np.array([1,2,3])

In [ ]:
A + v

In [ ]:
np.expand_dims(v, 1)

In [ ]:
A + np.expand_dims(v, 1)

### Общие правило broadcasting-a

<a href="https://tirinox.ru/numpy-broadcasting/">Ссылка на источник</a>

In [ ]:
a = np.ones((8, 1, 6, 1))
b = np.ones((7, 1, 5))

In [ ]:
a.shape

In [ ]:
b.shape

Сначала размеры (shape) массивов выстраивается друг над другом, выравнивая по правому краю. 

Cправа самая «глубокая» размерность.

```
A         (4d массив):  8 x 1 x 6 x 1
B         (3d массив):      7 x 1 x 5
```

Затем NumPy идет справа налево, поэлементно сравнивая каждый размер операндов. Два размера считаются совместимыми, если они равны или один из них равен единице. Если два размера несовместимы, бродкастинг не пройдет, возникнет ошибка.
ValueError: operands could not be broadcast together with shapes



Если слева не хватает размерности, то она автоматически расширяется единицей, это значит, что мы как будто бы оборачиваем массив в еще одни квадратные скобки. В нашем примере, у B не хватает одной размерности, так как он трехмерный вектор, превратим его в четырехмерный.

```
A         (4d массив):  8 x 1 x 6 x 1

B         (3d массив):      7 x 1 x 5
B'        (4d массив):  1 x 7 x 1 x 5
B' = [ B ] 
```

Мы видим, что в примере два массива полностью совместимы для бродкастинга – (8 над 1, 1 над 7, 6 над 1, 1 над 5): в каждом из столбиков есть единичка.

Теперь происходит самое интересное – там, где размеры это единицы происходит «копирование» каждого из таких измерений столько раз, чтобы размеры по этому измерению стали равны.

```
A         (4d массив):  8 x 1 x 6 x 1
B         (3d массив):      7 x 1 x 5
Результат (4d массив):  8 x 7 x 6 x 5
```

Как только все размерности выровнены путем «копирования», то можно делать любую операцию поэлементно. В итоге:

In [ ]:
(a + b).shape

### Вывод

Бродкастинг удобен, но может и навредить, потому что он не дает предупреждений, что массивы разного размера. Иными словами, можно умножить синий цвет на число крокодилов, и если повезло с размерностью крокодилов и цвета, то вы еще долго будете искать ошибку.

## Benchmark numpy array vs torch tensor

In [2]:
import torch.utils.benchmark as benchmark
from itertools import product
import numpy as np
import timeit
import torch

In [3]:
def numpy_matrix_multiplication(a, b):
    return a @ b


def torch_matrix_multiplication(a, b):
    return a @ b

In [4]:
results = []

sizes = []

for i in range(12):
    sizes.append(2**i)

for i in sizes:
    label = 'matrix multiplication'
    sub_label = f'array size {i}'
    x = np.random.rand(i,i)
    results.append(benchmark.Timer(
        stmt='numpy_matrix_multiplication(x, x)',
        setup='from __main__ import numpy_matrix_multiplication',
        globals={'x': x},    
        label=label,
        sub_label=sub_label,
        description='matrix multiplication numpy',
    ).timeit(10))
    
    results.append(benchmark.Timer(
        stmt='torch_matrix_multiplication(x, x)',
        setup='from __main__ import torch_matrix_multiplication',
        globals={'x': torch.from_numpy(x).to(torch.device("cuda"))},
        label=label,
        sub_label=sub_label,
        description='matrix multiplication torch',
    ).timeit(10))

In [5]:
compare = benchmark.Compare(results)
compare.print()

[------------------------------ matrix multiplication ------------------------------]
                       |  matrix multiplication numpy  |  matrix multiplication torch
1 threads: --------------------------------------------------------------------------
      array size 1     |                 4.1           |               37.2          
      array size 2     |                 9.8           |               37.7          
      array size 4     |                 9.7           |               39.0          
      array size 8     |                10.2           |               37.7          
      array size 16    |                 5.1           |               16.8          
      array size 32    |                11.3           |               16.2          
      array size 64    |                30.4           |               16.2          
      array size 128   |               135.4           |               43.0          
      array size 256   |               738.1          

In [7]:
del x